In [87]:
import numpy as np
import pandas as pd
import os
import seqlogo
import logomaker
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from PIL import Image
import io


产生seqlogo的图

In [88]:
def parse_meme_file(file_path):
    motifs = []
    current_motif = []
    reading_motif = False
    
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('MOTIF'):
                if current_motif:
                    motifs.append(pd.DataFrame(current_motif, columns=['A', 'C', 'G', 'T']))
                    current_motif = []
                reading_motif = True
            elif line.startswith('letter-probability matrix'):
                reading_motif = True
            elif reading_motif and line.strip() and not line.startswith('---'):
                values = [float(num) for num in line.split()]
                current_motif.append(values)
    
    # Add the last motif
    if current_motif:
        motifs.append(pd.DataFrame(current_motif, columns=['A', 'C', 'G', 'T']))
    
    return motifs


file_path = 'meme_file_multitask_TRASH_3L.txt'
motifs = parse_meme_file(file_path)

pdf_dir = 'tfmodisco_seqlogo_3L'
os.makedirs(pdf_dir, exist_ok=True)

for i, motif_df in enumerate(motifs):
    if not motif_df.empty:
        ppm = seqlogo.Ppm(motif_df.T)  
        png_path = os.path.join(pdf_dir, f'motif_{i+1}.png')
        seqlogo.seqlogo(ppm, ic_scale=True, format='png', size='medium', filename=png_path)
    else:
        print(f"Motif {i+1} is empty.")

输出number of seqlets

In [92]:
import h5py
import numpy as np

hdf5_results = h5py.File("/rugpfs/fs0/zhao_lab/scratch/xwu05/detect_peaks_scripts/run_model/multitask_results_X.hdf5","r")


metacluster_names = [x.decode("utf-8") for x in list(hdf5_results["metaclustering_results"]["all_metacluster_names"][:])]

patterns_list = []

# 遍历所有的metaclusters和它们的patterns
for metacluster_name in metacluster_names:
    metacluster_grp = hdf5_results["metacluster_idx_to_submetacluster_results"][metacluster_name]
    all_pattern_names = [x.decode("utf-8") for x in list(metacluster_grp["seqlets_to_patterns_result"]["patterns"]["all_pattern_names"][:])]

    for pattern_name in all_pattern_names:
        pattern = metacluster_grp["seqlets_to_patterns_result"]["patterns"][pattern_name]
        num_seqlets = len(pattern["seqlets_and_alnmts"]["seqlets"])
        
        # 将模式和seqlets数量添加到列表中
        patterns_list.append((metacluster_name + pattern_name, num_seqlets))


for item in patterns_list:
    print(f"Pattern: {item[0]}, Total seqlets: {item[1]}")

with open("patterns_seqlets_list_X.txt", "w") as f:
    for item in patterns_list:
        f.write(f"Pattern: {item[0]}, Total seqlets: {item[1]}\n")


Pattern: metacluster_0pattern_0, Total seqlets: 2426
Pattern: metacluster_0pattern_1, Total seqlets: 1913
Pattern: metacluster_0pattern_2, Total seqlets: 1711
Pattern: metacluster_0pattern_3, Total seqlets: 928
Pattern: metacluster_0pattern_4, Total seqlets: 642
Pattern: metacluster_0pattern_5, Total seqlets: 633
Pattern: metacluster_0pattern_6, Total seqlets: 612
Pattern: metacluster_0pattern_7, Total seqlets: 602
Pattern: metacluster_0pattern_8, Total seqlets: 539
Pattern: metacluster_0pattern_9, Total seqlets: 533
Pattern: metacluster_0pattern_10, Total seqlets: 532
Pattern: metacluster_0pattern_11, Total seqlets: 513
Pattern: metacluster_0pattern_12, Total seqlets: 464
Pattern: metacluster_0pattern_13, Total seqlets: 450
Pattern: metacluster_0pattern_14, Total seqlets: 420
Pattern: metacluster_0pattern_15, Total seqlets: 413
Pattern: metacluster_0pattern_16, Total seqlets: 413
Pattern: metacluster_0pattern_17, Total seqlets: 387
Pattern: metacluster_0pattern_18, Total seqlets: 364


In [93]:
pattern_seqlets = {}
with open("patterns_seqlets_list_X.txt", "r") as f:
    for line in f:
        parts = line.strip().split(", Total seqlets: ")
        if len(parts) == 2:
            pattern_name = parts[0].split(": ")[1]  # 提取模式名称
            pattern_seqlets[pattern_name] = parts[1]  # 将模式名称与seqlets数量存储在字典中
pattern_seqlets


{'metacluster_0pattern_0': '2426',
 'metacluster_0pattern_1': '1913',
 'metacluster_0pattern_2': '1711',
 'metacluster_0pattern_3': '928',
 'metacluster_0pattern_4': '642',
 'metacluster_0pattern_5': '633',
 'metacluster_0pattern_6': '612',
 'metacluster_0pattern_7': '602',
 'metacluster_0pattern_8': '539',
 'metacluster_0pattern_9': '533',
 'metacluster_0pattern_10': '532',
 'metacluster_0pattern_11': '513',
 'metacluster_0pattern_12': '464',
 'metacluster_0pattern_13': '450',
 'metacluster_0pattern_14': '420',
 'metacluster_0pattern_15': '413',
 'metacluster_0pattern_16': '413',
 'metacluster_0pattern_17': '387',
 'metacluster_0pattern_18': '364',
 'metacluster_0pattern_19': '357',
 'metacluster_0pattern_20': '351',
 'metacluster_0pattern_21': '321',
 'metacluster_0pattern_22': '315',
 'metacluster_0pattern_23': '299',
 'metacluster_0pattern_24': '263',
 'metacluster_0pattern_25': '251',
 'metacluster_0pattern_26': '236',
 'metacluster_0pattern_27': '170',
 'metacluster_0pattern_28':

In [94]:

folder_path = '/rugpfs/fs0/zhao_lab/scratch/xwu05/detect_peaks_scripts/run_model/tfmodisco_seqlogo_X'

for filename in os.listdir(folder_path):
    # 检查是否为PNG文件
    if filename.endswith('.png'):
        # 获取不带扩展名的文件名
        base_name = filename.split('.png')[0]
        # 检查文件名是否在字典中
        if base_name in pattern_seqlets:
            # 构造新文件名
            new_name = f"{base_name}_seqlets_{pattern_seqlets[base_name]}.png"
            # 构造完整的文件路径
            old_file = os.path.join(folder_path, filename)
            new_file = os.path.join(folder_path, new_name)
            # 重命名文件
            os.rename(old_file, new_file)
            print(f"Renamed {filename} to {new_name}")


Renamed metacluster_2pattern_9.png to metacluster_2pattern_9_seqlets_324.png
Renamed metacluster_2pattern_3.png to metacluster_2pattern_3_seqlets_814.png
Renamed metacluster_1pattern_10.png to metacluster_1pattern_10_seqlets_238.png
Renamed metacluster_2pattern_5.png to metacluster_2pattern_5_seqlets_684.png
Renamed metacluster_1pattern_12.png to metacluster_1pattern_12_seqlets_176.png
Renamed metacluster_1pattern_2.png to metacluster_1pattern_2_seqlets_2548.png
Renamed metacluster_3pattern_12.png to metacluster_3pattern_12_seqlets_149.png
Renamed metacluster_2pattern_6.png to metacluster_2pattern_6_seqlets_473.png
Renamed metacluster_4pattern_1.png to metacluster_4pattern_1_seqlets_679.png
Renamed metacluster_3pattern_1.png to metacluster_3pattern_1_seqlets_818.png
Renamed metacluster_0pattern_32.png to metacluster_0pattern_32_seqlets_66.png
Renamed metacluster_0pattern_29.png to metacluster_0pattern_29_seqlets_121.png
Renamed metacluster_1pattern_9.png to metacluster_1pattern_9_seqle